In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import re 

In [7]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q15-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JANUARY\q15-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\2025\JULY\q15-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [8]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                         Non-Null Count  Dtype  
---  ------                                                                                                                                                         --------------  -----  
 0   Response ID                                                                                                                                                    400 non-null    object 
 1   Online search (e.g. Google, Bing):BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?      40 non-null     float64
 2   Online shopping website:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on 

In [9]:
Germany.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:BOSCH:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:MIELE:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:ROWENTA:GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,GER2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GER2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform' column named 'Platform_cleaned'
unpivoted_GM['Platform_cleaned'] = unpivoted_GM['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_GM = unpivoted_GM[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_GM['Platform_cleaned'] = unpivoted_GM['Platform_cleaned'].str.replace(
    ":GM - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

      Response ID                                           Platform  \
23995     GER2396  None of the above:ROWENTA:GM - And on which of...   
23996     GER2397  None of the above:ROWENTA:GM - And on which of...   
23997     GER2398  None of the above:ROWENTA:GM - And on which of...   
23998     GER2399  None of the above:ROWENTA:GM - And on which of...   
23999     GER2400  None of the above:ROWENTA:GM - And on which of...   

                Platform_cleaned  Seen Y/N  
23995  None of the above:ROWENTA       NaN  
23996  None of the above:ROWENTA       NaN  
23997  None of the above:ROWENTA       NaN  
23998  None of the above:ROWENTA       NaN  
23999  None of the above:ROWENTA       NaN  


In [11]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_GM['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [12]:
count = unpivoted_GM['Seen Y/N'].value_counts()
print(count)

Seen Y/N
1.0     374
2.0     366
3.0     247
4.0     237
99.0     39
Name: count, dtype: int64


In [13]:
# Drop the original 'Platform_cleaned' column
unpivoted_GM = unpivoted_GM.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_GM = unpivoted_GM[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_GM['Seen Y/N'] = np.where(unpivoted_GM['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_GM['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_GM['Seen Y/N'].isna(), 0, unpivoted_GM['Seen Y/N'])))

In [14]:
unpivoted_GM.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,GER2001,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
1,GER2002,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
2,GER2003,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
3,GER2004,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH
4,GER2005,"Online search (e.g. Google, Bing):BOSCH:GM - A...","Online search (e.g. Google, Bing)",0.0,BOSCH


In [15]:
count = unpivoted_GM['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22737
1.0      1224
99.0       39
Name: count, dtype: int64


Brand
BOSCH        2000
MIELE        2000
SIEMENS      2000
AEG          2000
SAMSUNG      2000
BAUKNECHT    2000
PHILIPS      2000
LG           2000
DYSON        2000
NEFF         2000
HAIER        2000
ROWENTA      2000
Name: count, dtype: int64


Total number of rows: 24000


### Sweden

In [16]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Online search (e.g. Google, Bing):ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?  41 non-null     float64
 2   Online shopping website:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or sho

In [17]:
Sweden.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:ELECTROLUX:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:AEG:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:ROBOROCK:SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,SWE2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWE2002,1.0,2.0,NaN,4.0,NaN,1.0,2.0,NaN,4.0,...,1.0,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_SW['Platform_cleaned'] = unpivoted_SW['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_SW = unpivoted_SW[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_SW['Platform_cleaned'] = unpivoted_SW['Platform_cleaned'].str.replace(
    ":SW - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

      Response ID                                           Platform  \
23995     SWE2396  None of the above:ROBOROCK:SW - And on which o...   
23996     SWE2397  None of the above:ROBOROCK:SW - And on which o...   
23997     SWE2398  None of the above:ROBOROCK:SW - And on which o...   
23998     SWE2399  None of the above:ROBOROCK:SW - And on which o...   
23999     SWE2400  None of the above:ROBOROCK:SW - And on which o...   

                 Platform_cleaned  Seen Y/N  
23995  None of the above:ROBOROCK       NaN  
23996  None of the above:ROBOROCK       NaN  
23997  None of the above:ROBOROCK       NaN  
23998  None of the above:ROBOROCK       NaN  
23999  None of the above:ROBOROCK       NaN  


In [19]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_SW['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [20]:
count = unpivoted_SW['Seen Y/N'].value_counts()
print(count)

Seen Y/N
1.0     373
2.0     319
3.0     282
4.0     202
99.0     50
Name: count, dtype: int64


In [21]:
# Drop the original 'Platform_cleaned' column
unpivoted_SW = unpivoted_SW.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_SW = unpivoted_SW[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_SW['Seen Y/N'] = np.where(unpivoted_SW['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_SW['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_SW['Seen Y/N'].isna(), 0, unpivoted_SW['Seen Y/N'])))

In [22]:
unpivoted_SW.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,SWE2001,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
1,SWE2002,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",1.0,ELECTROLUX
2,SWE2003,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
3,SWE2004,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX
4,SWE2005,"Online search (e.g. Google, Bing):ELECTROLUX:S...","Online search (e.g. Google, Bing)",0.0,ELECTROLUX


In [23]:
count = unpivoted_SW['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22774
1.0      1176
99.0       50
Name: count, dtype: int64


Brand
ELECTROLUX    2000
BOSCH         2000
SAMSUNG       2000
MIELE         2000
PHILIPS       2000
SIEMENS       2000
CYLINDA       2000
WHIRLPOOL     2000
LG            2000
DYSON         2000
AEG           2000
ROBOROCK      2000
Name: count, dtype: int64


Total number of rows: 24000


### Italy

In [24]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Online search (e.g. Google, Bing):SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     65 non-null     float64
 2   Online shopping website:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [25]:
Italy.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SIEMENS:IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,ITA2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ITA2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_IT['Platform_cleaned'] = unpivoted_IT['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_IT = unpivoted_IT[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_IT['Platform_cleaned'] = unpivoted_IT['Platform_cleaned'].str.replace(
    ":IT - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

      Response ID                                           Platform  \
23995     ITA2396  None of the above:SIEMENS:IT - And on which of...   
23996     ITA2397  None of the above:SIEMENS:IT - And on which of...   
23997     ITA2398  None of the above:SIEMENS:IT - And on which of...   
23998     ITA2399  None of the above:SIEMENS:IT - And on which of...   
23999     ITA2400  None of the above:SIEMENS:IT - And on which of...   

                Platform_cleaned  Seen Y/N  
23995  None of the above:SIEMENS       NaN  
23996  None of the above:SIEMENS       NaN  
23997  None of the above:SIEMENS       NaN  
23998  None of the above:SIEMENS       NaN  
23999  None of the above:SIEMENS       NaN  


In [27]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_IT['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [28]:
count = unpivoted_IT['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     481
1.0     413
3.0     319
4.0     262
99.0     19
Name: count, dtype: int64


In [29]:
# Drop the original 'Platform_cleaned' column
unpivoted_IT = unpivoted_IT.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_IT = unpivoted_IT[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_IT['Seen Y/N'] = np.where(unpivoted_IT['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_IT['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_IT['Seen Y/N'].isna(), 0, unpivoted_IT['Seen Y/N'])))

In [30]:
unpivoted_IT.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,ITA2001,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,ITA2002,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,ITA2003,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,ITA2004,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,ITA2005,"Online search (e.g. Google, Bing):SAMSUNG:IT -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [31]:
count = unpivoted_IT['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22506
1.0      1475
99.0       19
Name: count, dtype: int64


Brand
SAMSUNG       2000
BOSCH         2000
WHIRLPOOL     2000
LG            2000
DYSON         2000
MIELE         2000
ELECTROLUX    2000
HOTPOINT      2000
BEKO          2000
AEG           2000
HAIER         2000
SIEMENS       2000
Name: count, dtype: int64


Total number of rows: 24000


### UK

In [32]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                        Non-Null Count  Dtype  
---  ------                                                                                                                                                        --------------  -----  
 0   Response ID                                                                                                                                                   400 non-null    object 
 1   Online search (e.g. Google, Bing):SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?   55 non-null     float64
 2   Online shopping website:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on li

In [33]:
UK.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LG:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SIEMENS:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:NEFF:UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,UK2001,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,UK2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_UK['Platform_cleaned'] = unpivoted_UK['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_UK = unpivoted_UK[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_UK['Platform_cleaned'] = unpivoted_UK['Platform_cleaned'].str.replace(
    ":UK - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

      Response ID                                           Platform  \
23995      UK2396  None of the above:NEFF:UK - And on which of th...   
23996      UK2397  None of the above:NEFF:UK - And on which of th...   
23997      UK2398  None of the above:NEFF:UK - And on which of th...   
23998      UK2399  None of the above:NEFF:UK - And on which of th...   
23999      UK2400  None of the above:NEFF:UK - And on which of th...   

             Platform_cleaned  Seen Y/N  
23995  None of the above:NEFF       NaN  
23996  None of the above:NEFF      99.0  
23997  None of the above:NEFF       NaN  
23998  None of the above:NEFF       NaN  
23999  None of the above:NEFF       NaN  


In [35]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_UK['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [36]:
count = unpivoted_UK['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     484
1.0     426
4.0     312
3.0     261
99.0     75
Name: count, dtype: int64


In [37]:
# Drop the original 'Platform_cleaned' column
unpivoted_UK = unpivoted_UK.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_UK = unpivoted_UK[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_UK['Seen Y/N'] = np.where(unpivoted_UK['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_UK['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_UK['Seen Y/N'].isna(), 0, unpivoted_UK['Seen Y/N'])))

In [38]:
unpivoted_UK.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,UK2001,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,UK2002,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,UK2003,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,UK2004,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,UK2005,"Online search (e.g. Google, Bing):SAMSUNG:UK -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [39]:
count = unpivoted_UK['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22442
1.0      1483
99.0       75
Name: count, dtype: int64


Brand
SAMSUNG     2000
LG          2000
HOTPOINT    2000
BOSCH       2000
DYSON       2000
BEKO        2000
SHARK       2000
ZANUSSI     2000
MIELE       2000
AEG         2000
SIEMENS     2000
NEFF        2000
Name: count, dtype: int64


Total number of rows: 24000


### Poland

In [40]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Online search (e.g. Google, Bing):SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     47 non-null     float64
 2   Online shopping website:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [41]:
Poland.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:MIELE:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:AEG:PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,POL2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POL2002,1.0,2.0,3.0,NaN,NaN,1.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_PO['Platform_cleaned'] = unpivoted_PO['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_PO = unpivoted_PO[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_PO['Platform_cleaned'] = unpivoted_PO['Platform_cleaned'].str.replace(
    ":PO - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

      Response ID                                           Platform  \
23995     POL2396  None of the above:AEG:PO - And on which of the...   
23996     POL2397  None of the above:AEG:PO - And on which of the...   
23997     POL2398  None of the above:AEG:PO - And on which of the...   
23998     POL2399  None of the above:AEG:PO - And on which of the...   
23999     POL2400  None of the above:AEG:PO - And on which of the...   

            Platform_cleaned  Seen Y/N  
23995  None of the above:AEG       NaN  
23996  None of the above:AEG       NaN  
23997  None of the above:AEG       NaN  
23998  None of the above:AEG       NaN  
23999  None of the above:AEG       NaN  


In [43]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_PO['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [44]:
count = unpivoted_PO['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     398
1.0     382
3.0     305
4.0     289
99.0     26
Name: count, dtype: int64


In [45]:
# Drop the original 'Platform_cleaned' column
unpivoted_PO = unpivoted_PO.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_PO = unpivoted_PO[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_PO['Seen Y/N'] = np.where(unpivoted_PO['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_PO['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_PO['Seen Y/N'].isna(), 0, unpivoted_PO['Seen Y/N'])))

In [46]:
unpivoted_PO.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,POL2001,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,POL2002,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
2,POL2003,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,POL2004,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,POL2005,"Online search (e.g. Google, Bing):SAMSUNG:PO -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [47]:
count = unpivoted_PO['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22600
1.0      1374
99.0       26
Name: count, dtype: int64


Brand
SAMSUNG       2000
BOSCH         2000
ELECTROLUX    2000
BEKO          2000
AMICA         2000
PHILIPS       2000
WHIRLPOOL     2000
LG            2000
DYSON         2000
HAIER         2000
MIELE         2000
AEG           2000
Name: count, dtype: int64


Total number of rows: 24000


### Thailand

In [48]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Online search (e.g. Google, Bing):SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     114 non-null    float64
 2   Online shopping website:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [49]:
Thailand.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LG:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:BOSCH:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:DYSON:TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,TH2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TH2002,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN


In [50]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_TH['Platform_cleaned'] = unpivoted_TH['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_TH = unpivoted_TH[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_TH['Platform_cleaned'] = unpivoted_TH['Platform_cleaned'].str.replace(
    ":TH - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

      Response ID                                           Platform  \
23995      TH2396  None of the above:DYSON:TH - And on which of t...   
23996      TH2397  None of the above:DYSON:TH - And on which of t...   
23997      TH2398  None of the above:DYSON:TH - And on which of t...   
23998      TH2399  None of the above:DYSON:TH - And on which of t...   
23999      TH2400  None of the above:DYSON:TH - And on which of t...   

              Platform_cleaned  Seen Y/N  
23995  None of the above:DYSON       NaN  
23996  None of the above:DYSON       NaN  
23997  None of the above:DYSON       NaN  
23998  None of the above:DYSON       NaN  
23999  None of the above:DYSON       NaN  


In [51]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Platform_cleaned'
for entry in unpivoted_TH['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [52]:
count = unpivoted_TH['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     949
1.0     895
4.0     661
3.0     656
99.0     23
Name: count, dtype: int64


In [53]:
# Drop the original 'Platform_cleaned' column
unpivoted_TH = unpivoted_TH.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_TH = unpivoted_TH[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_TH['Seen Y/N'] = np.where(unpivoted_TH['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_TH['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_TH['Seen Y/N'].isna(), 0, unpivoted_TH['Seen Y/N'])))

In [54]:
unpivoted_TH.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,TH2001,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,TH2002,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,TH2003,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
3,TH2004,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,TH2005,"Online search (e.g. Google, Bing):SAMSUNG:TH -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [55]:
count = unpivoted_TH['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     20816
1.0      3161
99.0       23
Name: count, dtype: int64


Brand
SAMSUNG       2000
LG            2000
MITSUBISHI    2000
ELECTROLUX    2000
PANASONIC     2000
SHARP         2000
PHILIPS       2000
HITACHI       2000
HAIER         2000
TEFAL         2000
BOSCH         2000
DYSON         2000
Name: count, dtype: int64


Total number of rows: 24000


### Vietnam

In [56]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Online search (e.g. Google, Bing):SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?     92 non-null     float64
 2   Online shopping website:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shoppi

In [57]:
Vietnam.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:PANASONIC:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:BOSCH:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:MIDEA:VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,VN02001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VN02002,1.0,2.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_VN['Platform_cleaned'] = unpivoted_VN['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_VN = unpivoted_VN[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_VN['Platform_cleaned'] = unpivoted_VN['Platform_cleaned'].str.replace(
    ":VN - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

      Response ID                                           Platform  \
23995     VN02396  None of the above:MIDEA:VN - And on which of t...   
23996     VN02397  None of the above:MIDEA:VN - And on which of t...   
23997     VN02398  None of the above:MIDEA:VN - And on which of t...   
23998     VN02399  None of the above:MIDEA:VN - And on which of t...   
23999     VN02400  None of the above:MIDEA:VN - And on which of t...   

              Platform_cleaned  Seen Y/N  
23995  None of the above:MIDEA       NaN  
23996  None of the above:MIDEA       NaN  
23997  None of the above:MIDEA       NaN  
23998  None of the above:MIDEA       NaN  
23999  None of the above:MIDEA       NaN  


In [59]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_VN['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [60]:
count = unpivoted_VN['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     846
1.0     780
4.0     597
3.0     549
99.0     18
Name: count, dtype: int64


In [61]:
# Drop the original 'Platform_cleaned' column
unpivoted_VN = unpivoted_VN.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_VN = unpivoted_VN[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_VN['Seen Y/N'] = np.where(unpivoted_VN['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_VN['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_VN['Seen Y/N'].isna(), 0, unpivoted_VN['Seen Y/N'])))

In [62]:
unpivoted_VN.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,VN02001,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,VN02002,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
2,VN02003,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG
3,VN02004,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,VN02005,"Online search (e.g. Google, Bing):SAMSUNG:VN -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG


In [63]:
count = unpivoted_VN['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     21210
1.0      2772
99.0       18
Name: count, dtype: int64


Brand
SAMSUNG       2000
PANASONIC     2000
LG            2000
TOSHIBA       2000
ELECTROLUX    2000
SHARP         2000
PHILIPS       2000
HITACHI       2000
XIAOMI        2000
AQUA          2000
BOSCH         2000
MIDEA         2000
Name: count, dtype: int64


Total number of rows: 24000


### Australia

In [64]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                             Non-Null Count  Dtype  
---  ------                                                                                                                                                             --------------  -----  
 0   Response ID                                                                                                                                                        400 non-null    object 
 1   Online search (e.g. Google, Bing):SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?        94 non-null     float64
 2   Online shopping website:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searchi

In [65]:
Australia.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SAMSUNG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:HAIER:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:AEG:AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,AUS2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AUS2002,NaN,2.0,3.0,NaN,NaN,1.0,2.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_AU['Platform_cleaned'] = unpivoted_AU['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_AU = unpivoted_AU[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_AU['Platform_cleaned'] = unpivoted_AU['Platform_cleaned'].str.replace(
    ":AU - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

      Response ID                                           Platform  \
23995     AUS2396  None of the above:AEG:AU - And on which of the...   
23996     AUS2397  None of the above:AEG:AU - And on which of the...   
23997     AUS2398  None of the above:AEG:AU - And on which of the...   
23998     AUS2399  None of the above:AEG:AU - And on which of the...   
23999     AUS2400  None of the above:AEG:AU - And on which of the...   

            Platform_cleaned  Seen Y/N  
23995  None of the above:AEG       NaN  
23996  None of the above:AEG       NaN  
23997  None of the above:AEG       NaN  
23998  None of the above:AEG       NaN  
23999  None of the above:AEG       NaN  


In [67]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_AU['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [68]:
count = unpivoted_AU['Seen Y/N'].value_counts()
print(count)

Seen Y/N
2.0     803
1.0     733
4.0     570
3.0     538
99.0     21
Name: count, dtype: int64


In [69]:
# Drop the original 'Platform_cleaned' column
unpivoted_AU = unpivoted_AU.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_AU = unpivoted_AU[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_AU['Seen Y/N'] = np.where(unpivoted_AU['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_AU['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_AU['Seen Y/N'].isna(), 0, unpivoted_AU['Seen Y/N'])))

In [70]:
unpivoted_AU.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,AUS2001,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
1,AUS2002,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
2,AUS2003,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
3,AUS2004,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",0.0,SAMSUNG
4,AUS2005,"Online search (e.g. Google, Bing):SAMSUNG:AU -...","Online search (e.g. Google, Bing)",1.0,SAMSUNG


In [71]:
count = unpivoted_AU['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     21335
1.0      2644
99.0       21
Name: count, dtype: int64


Brand
SAMSUNG         2000
LG              2000
DYSON           2000
WESTINGHOUSE    2000
BOSCH           2000
HISENSE         2000
MIELE           2000
SMEG            2000
ELECTROLUX      2000
HAIER           2000
AEG             2000
Name: count, dtype: int64


Total number of rows: 24000


In [72]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 2000


In [73]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [74]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          2000
LG               2000
DYSON            2000
WESTINGHOUSE     2000
FISHER&PAYKEL    2000
BOSCH            2000
HISENSE          2000
MIELE            2000
SMEG             2000
ELECTROLUX       2000
HAIER            2000
AEG              2000
Name: count, dtype: int64


Total number of rows: 24000


### Switzerland

In [75]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 61 columns):
 #   Column                                                                                                                                                          Non-Null Count  Dtype  
---  ------                                                                                                                                                          --------------  -----  
 0   Response ID                                                                                                                                                     400 non-null    object 
 1   Online search (e.g. Google, Bing):V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?       33 non-null     float64
 2   Online shopping website:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping

In [76]:
Switzerland.head(2) #initial dataset

,Response ID,"Online search (e.g. Google, Bing):V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:V ZUG:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:ELECTROLUX:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,...,"Online search (e.g. Google, Bing):LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:LIEBHERR:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,"Online search (e.g. Google, Bing):SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?",Online shopping website:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Price comparison site:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,Product reviews:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?,None of the above:SCHULTHESS:SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?
0,SWIT2001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,SWIT2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# Unpivot the DataFrame, renaming 'Variable' to 'Platform'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Platform', value_name='Seen Y/N')

# Create a duplicate of the 'Platform_cleaned' column named 'Platform'
unpivoted_SZ['Platform_cleaned'] = unpivoted_SZ['Platform']

# Rearrange columns to place 'Platform_cleaned' next to 'Source'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N']]

# Remove the unwanted suffix from the 'Platform_cleaned' column only
unpivoted_SZ['Platform_cleaned'] = unpivoted_SZ['Platform_cleaned'].str.replace(
    ":SZ - And on which of these types of sites have you seen these brands recently when searching or shopping on line?", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

      Response ID                                           Platform  \
23995    SWIT2396  None of the above:SCHULTHESS:SZ - And on which...   
23996    SWIT2397  None of the above:SCHULTHESS:SZ - And on which...   
23997    SWIT2398  None of the above:SCHULTHESS:SZ - And on which...   
23998    SWIT2399  None of the above:SCHULTHESS:SZ - And on which...   
23999    SWIT2400  None of the above:SCHULTHESS:SZ - And on which...   

                   Platform_cleaned  Seen Y/N  
23995  None of the above:SCHULTHESS       NaN  
23996  None of the above:SCHULTHESS       NaN  
23997  None of the above:SCHULTHESS       NaN  
23998  None of the above:SCHULTHESS       NaN  
23999  None of the above:SCHULTHESS       NaN  


In [78]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'cleaned_source'
for entry in unpivoted_SZ['Platform_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [79]:
count = unpivoted_SZ['Seen Y/N'].value_counts()
print(count)

Seen Y/N
1.0     308
2.0     278
3.0     216
4.0     211
99.0     48
Name: count, dtype: int64


In [80]:
# Drop the original 'Platform_cleaned' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['Platform_cleaned'])

# Rename 'Ad_Type' to 'Platform_cleaned'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'Platform_cleaned'})

# Rearrange columns to place 'Platform_cleaned' next to 'Platform'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Platform', 'Platform_cleaned', 'Seen Y/N', 'Brand']]

# Replace values in the 'Seen Y/N' column
unpivoted_SZ['Seen Y/N'] = np.where(unpivoted_SZ['Seen Y/N'].isin([1, 2, 3, 4]), 1, 
                        np.where(unpivoted_SZ['Seen Y/N'] == 99, 99,  
                        np.where(unpivoted_SZ['Seen Y/N'].isna(), 0, unpivoted_SZ['Seen Y/N'])))

In [81]:
unpivoted_SZ.head()

,Response ID,Platform,Platform_cleaned,Seen Y/N,Brand
0,SWIT2001,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
1,SWIT2002,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
2,SWIT2003,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
3,SWIT2004,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None
4,SWIT2005,"Online search (e.g. Google, Bing):V ZUG:SZ - A...","Online search (e.g. Google, Bing)",0.0,None


In [82]:
count = unpivoted_SZ['Seen Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Seen Y/N
0.0     22939
1.0      1013
99.0       48
Name: count, dtype: int64


Brand
ELECTROLUX    2000
MIELE         2000
DYSON         2000
BOSCH         2000
SAMSUNG       2000
SIEMENS       2000
PHILIPS       2000
BAUKNECHT     2000
AEG           2000
LIEBHERR      2000
SCHULTHESS    2000
Name: count, dtype: int64


Total number of rows: 24000


In [83]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 2000


In [84]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [85]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         2000
ELECTROLUX    2000
MIELE         2000
DYSON         2000
BOSCH         2000
SAMSUNG       2000
SIEMENS       2000
PHILIPS       2000
BAUKNECHT     2000
AEG           2000
LIEBHERR      2000
SCHULTHESS    2000
Name: count, dtype: int64


Total number of rows: 24000


### Concat and save them in another csv file

In [86]:
# Concatenate the melted DataFrames
combined_q15 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)

# combined_q17.dropna(subset=['Response ID'], inplace=True)

combined_q15 = combined_q15.sort_values(by='Response ID', ascending=True)

# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q15.xlsx', float_format='%.0f', index=False)
# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q15.xlsx', float_format='%.0f', index=False)
# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q15.xlsx', float_format='%.0f', index=False)
# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q15.xlsx', float_format='%.0f', index=False)
# combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JANUARY\combined_q15.xlsx', float_format='%.0f', index=False)
combined_q15.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\2025\JULY\combined_q15.xlsx', float_format='%.0f', index=False)

print(combined_q15.head(2))

       Response ID                                           Platform  \
168000     AUS2001  Online search (e.g. Google, Bing):SAMSUNG:AU -...   
177600     AUS2001  None of the above:FISHER&PAYKEL:AU - And on wh...   

                         Platform_cleaned  Seen Y/N          Brand  
168000  Online search (e.g. Google, Bing)       0.0        SAMSUNG  
177600                  None of the above       0.0  FISHER&PAYKEL  


In [87]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q15['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [88]:
combined_q15.info()

<class 'pandas.core.frame.DataFrame'>
Index: 216000 entries, 168000 to 155999
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Response ID       216000 non-null  object 
 1   Platform          216000 non-null  object 
 2   Platform_cleaned  216000 non-null  object 
 3   Seen Y/N          216000 non-null  float64
 4   Brand             216000 non-null  object 
dtypes: float64(1), object(4)
memory usage: 9.9+ MB


In [89]:
count = combined_q15['Brand'].value_counts()
print(count)

Brand
SAMSUNG          18000
BOSCH            18000
DYSON            16000
LG               16000
AEG              14000
ELECTROLUX       14000
MIELE            14000
PHILIPS          12000
HAIER            10000
SIEMENS          10000
WHIRLPOOL         6000
BEKO              6000
BAUKNECHT         4000
SHARP             4000
PANASONIC         4000
HOTPOINT          4000
NEFF              4000
HITACHI           4000
FISHER&PAYKEL     2000
WESTINGHOUSE      2000
ROWENTA           2000
AMICA             2000
SMEG              2000
HISENSE           2000
ROBOROCK          2000
SCHULTHESS        2000
LIEBHERR          2000
V ZUG             2000
CYLINDA           2000
MITSUBISHI        2000
TEFAL             2000
SHARK             2000
ZANUSSI           2000
MIDEA             2000
TOSHIBA           2000
XIAOMI            2000
AQUA              2000
Name: count, dtype: int64
